In [1]:
%matplotlib inline
%precision 16
import numpy
import matplotlib.pyplot as plt

Test for matrix construction

In [2]:
M = 5
N = 5

In [3]:
K     = 100

S_max = 200
S_min = 0
v_max = 1.0

alpha = 1.0 #???
beta  = 1.0 #???
c1    = numpy.arcsinh( (S_max - K)/alpha )
c2    = numpy.arcsinh( (S_min - K)/alpha )
d1     = numpy.arcsinh( v_max/beta )

# Set Coefficient 

rho   = 0.6
sigma = 0.04
r     = 0.01
q     = 0.04
kappa = 3
theta = 1.0 #???

# Set grid
# Domain (t x \xi x \eta) : 
#
# [0, T] x [0, 1] x [0,1]
#  xi[j] = \Delta xi  * j, j = 0, ..., N+1
# eta[j] = \Delta eta * j, j = 0, ..., M+1
#   t[j] = \Delta t   * j, j = 0, ..., L-1
#

t_final = 1.0
N = 100
M = 100
L = 100

xi  = numpy.linspace(0, 1, N+2) 
eta = numpy.linspace(0, 1, M+2)
t   = numpy.linspace(0, t_final, L)

delta_xi  =  xi[1] -  xi[0]
delta_eta = eta[1] - eta[0]
delta_t   =   t[1] -   t[0]

# Some function needed because of the change of variables
S       = lambda xi: K + numpy.sinh( c1*xi + c2*(1-xi) )
v       = lambda eta: beta*numpy.sinh( d1*eta )
dSdxi   = lambda xi: numpy.cosh( c1*xi + c2*(1-xi) ) * (c1 - c2)
d2Sdxi2 = lambda xi: numpy.sinh( c1*xi + c2*(1-xi) ) * (c1 - c2)**2
dvdeta  = lambda eta: numpy.cosh( d1*eta ) * d1 * beta
d2vdeta2= lambda eta: numpy.cosh( d1*eta ) * d1**2 * beta

xi  =  xi[1:-1]
eta = eta[1:-1]

In [4]:
v(eta[1])

0.0174538283735313

In [5]:
a = lambda i,j: delta_t*rho*sigma/(4.0*delta_xi*delta_eta)*v(eta[j])*S(xi[i])/(dSdxi(xi[i]) * dvdeta(eta[j]))
b = lambda i,j: sigma**2 * delta_t/(2.0*delta_eta**2)*v(eta[j])/dvdeta(eta[i])**2 - \
                delta_t/(2.0*delta_eta)*(kappa*(theta-v(eta[j]))/dvdeta(eta[j]) - 0.5*sigma**2*v(eta[j])*d2vdeta2(eta[j])/dvdeta(eta[j])**3)
c = lambda i,j: delta_t/delta_xi**2*v(eta[j])*S(xi[i])**2/dSdxi(xi[i])**2 - \
                delta_t/(2.0*delta_xi)*((r-q)*S(xi[i])/dSdxi(xi[i]) - 0.5*v(eta[j])*S(xi[i])**2*d2Sdxi2(xi[i])/dSdxi(xi[i])**3)
d = lambda i,j: 1+r*delta_t+delta_t/delta_xi**2*v(eta[j])*S(xi[i])**2/dSdxi(xi[i])**2+sigma**2*delta_t/delta_eta**2*v(eta[j])/dvdeta(eta[i])**2
e = lambda i,j: delta_t/delta_xi**2*v(eta[j])*S(xi[i])**2/dSdxi(xi[i])**2 + \
                delta_t/(2*delta_xi)*((r-q)*S(xi[i])/dSdxi(xi[i]) - 0.5*v(eta[j])*S(xi[i])**2*d2Sdxi2(xi[i])/dSdxi(xi[i])**3)
f = lambda i,j: sigma**2*delta_t/(2*delta_eta**2)*v(eta[j])/dvdeta(eta[i])**2 + \
                delta_t/(2.0*delta_eta)*(kappa*(theta - v(eta[j]))/dvdeta(eta[j]) - 0.5*sigma**2*v(eta[j])*d2vdeta2(eta[j])/dvdeta(eta[j])**3)
A = numpy.zeros([M*N, M*N])


In [8]:
rhs = numpy.zeros(N*M)
# Boundary condition 
# For bottom bc xi = 0 
_l = 2.0 * dSdxi(xi[0]) / ( dSdxi(xi[0]) + 0.5 * delta_xi * d2Sdxi2(xi[0]) )
l_ = - ( dSdxi(xi[0]) - 0.5* delta_xi * d2Sdxi2(xi[0]) ) / ( dSdxi(xi[0]) + 0.5* delta_xi* d2Sdxi2(xi[0]) )

# For upper bc xi = 1
_r = - ( dSdxi(xi[-1]) + 0.5 * delta_xi * d2Sdxi2(xi[-1]) ) / ( dSdxi(xi[-1]) - 0.5 * delta_xi * d2Sdxi2(xi[-1]) )
r_ = 2.0 * dSdxi(xi[-1]) / ( dSdxi(xi[-1]) - 0.5 * delta_xi * d2Sdxi2(xi[-1]) )

In [9]:
for irow in range(M*N):
    # U_hat[irow] == U_interior[i,j]
    i = irow/M
    j = numpy.mod(irow, N)

    if( i == 0 ):
        if (j == 0):
            A[irow, irow + M]     = -e(i,j)
            A[irow, irow + M + 1] = -a(i,j)
            A[irow, irow]         =  d(i,j)
            A[irow, irow + 1]     = -f(i,j)
        elif (j == M-1):
            A[irow, irow + M - 1] =  a(i,j)
            A[irow, irow + M]     = -e(i,j)
            A[irow, irow - 1]     = -b(i,j)
            A[irow, irow]         =  d(i,j)
            # Use bc for eta = 1
            rhs[irow] = -a(i,j)*S(xi[i-1]) + f(i,j)*S(xi[i]) + a(i,j)*S(xi[i+1])
        else: # Add term for bc condition
            A[irow, irow + M - 1] =  a(i,j) - a(i,j)*l_
            A[irow, irow + M]     = -e(i,j) - c(i,j)*l_
            A[irow, irow + M + 1] = -a(i,j) + a(i,j)*l_
            A[irow, irow - 1]     = -b(i,j) - a(i,j)*_l
            A[irow, irow]         =  d(i,j) - c(i,j)*_l
            A[irow, irow + 1]     = -f(i,j) + a(i,j)*_l
    elif (i == N-1):
        if (j == 0):
            A[irow, irow - M ]    = -c(i,j)
            A[irow, irow - M + 1] =  a(i,j)
            A[irow, irow]         =  d(i,j)
            A[irow, irow + 1]     = -f(i,j)
        elif (j == M-1):
            A[irow, irow - M - 1] = -a(i,j) 
            A[irow, irow - M ]    = -c(i,j)
            A[irow, irow - 1]     = -b(i,j)
            A[irow, irow]         =  d(i,j)
            # Use bc for eta = 1
            rhs[irow] = -a(i,j)*S(xi[i-1]) + f(i,j)*S(xi[i]) + a(i,j)*S(xi[i+1])
        else:
            A[irow, irow - M - 1] = -a(i,j) + a(i,j)*_r
            A[irow, irow - M ]    = -c(i,j) - e(i,j)*_r
            A[irow, irow - M + 1] =  a(i,j) - a(i,j)*_r
            A[irow, irow - 1]     = -b(i,j) + a(i,j)*r_
            A[irow, irow]         =  d(i,j) - e(i,j)*r_
            A[irow, irow + 1]     = -f(i,j) - a(i,j)*r_
    else:
        if (j == 0):   
            A[irow, irow - M ]    = -e(i,j)
            A[irow, irow - M + 1] = -a(i,j)
            A[irow, irow]         =  d(i,j)
            A[irow, irow + 1]     = -f(i,j)
            A[irow, irow + M]     = -c(i,j)
            A[irow, irow + M + 1] =  a(i,j)
        elif (j == M-1):
            A[irow, irow - M - 1] = -a(i,j) 
            A[irow, irow - M ]    = -c(i,j)
            A[irow, irow - 1]     = -b(i,j)
            A[irow, irow]         =  d(i,j)
            A[irow, irow + M - 1] =  a(i,j)
            A[irow, irow + M]     = -e(i,j)
            rhs[irow] = -a(i,j)*S(xi[i-1]) + f(i,j)*S(xi[i]) + a(i,j)*S(xi[i+1])
        else:
            A[irow, irow - M - 1] = -a(i,j) 
            A[irow, irow - M ]    = -c(i,j)
            A[irow, irow - M + 1] =  a(i,j)
            A[irow, irow - 1]     = -b(i,j)
            A[irow, irow]         =  d(i,j)
            A[irow, irow + 1]     = -f(i,j)
            A[irow, irow + M - 1] =  a(i,j)
            A[irow, irow + M]     = -e(i,j)
            A[irow, irow + M + 1] = -a(i,j)

In [13]:
#fig = plt.figure()
#plt.spy(A[0:200,0:200])
for i in range(len(rhs)):
    print rhs[i]

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.45178038973
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.